In [65]:
import networkx as nx
# create a DAG
g = nx.DiGraph()
# add nl[j]s and edges to the DAG
g.add_edges_from([("A", "D"), ("B", "D"), ("B", "E"), ("B", "G"), ("C", "E"),("C", "F"), ("D", "I"), ("E", "G"),("E", "H"), ("F", "H"), ("G", "I"), ("G","J"), ("H", "J")])
nx.set_node_attributes(g, {"A": "PI", "B": "PI", "C":"PI", "D":"AND2", "E":"XOR2", "F":"INV", "G":"AND2", "H":"AND2", "I":"OR2", "J":"OR2"},name="gateType")
print(g.nodes(data=True))
# sort the nodes in topological order
nl = list(nx.topological_sort(g))
print('Nodes in topological order',nl)
# find children of a node
print('Successors of node B', list(g.successors("B")))
# find parents of a node
print('Parents of node H', list(g.predecessors("H")))
['C', 'F', 'A', 'B', 'D', 'E', 'G', 'H', 'I', 'J']

[('A', {'gateType': 'PI'}), ('D', {'gateType': 'AND2'}), ('B', {'gateType': 'PI'}), ('E', {'gateType': 'XOR2'}), ('G', {'gateType': 'AND2'}), ('C', {'gateType': 'PI'}), ('F', {'gateType': 'INV'}), ('I', {'gateType': 'OR2'}), ('H', {'gateType': 'AND2'}), ('J', {'gateType': 'OR2'})]


TypeError: 'NodeDataView' object is not callable

# Getting the input from the netlist file

In [1]:

# The program will throw in an error if there isn't exactly 2 arguments in the commandline.
# if len(argv)!=2:
# 	print("Please provide the correct 2 arguments in the commandline.")
# 	exit()
import networkx as nx

class Component:
	def __init__(self,name,g_type,input,output):
		self.name = name
		self.g_type = g_type
		self.input = input
		self.output = output
l = []
	# Opening the file mentioned in the commandline.
with open("c17.net") as f:
    lines = f.readlines()
    for line in lines:
        name,g_type,*input= line.split()
        output = line.split()[-1]
        Gate = Component(name,g_type,input,output)
        l.append(Gate)

edges = []
g = nx.DiGraph()
for gate in l:
    if len(gate.input) == 3:
        edges.append((gate.input[0],gate.output))
        edges.append((gate.input[1],gate.output))
    else:
        edges.append((gate.input[0],gate.output))

g.add_edges_from(edges)
value = []
with open("c17.inputs") as file:
    lineIn = file.readlines()
    pi = lineIn[0].split()
    for i in range(len(lineIn)-1):
        value.append({})
        for j in range(len(pi)):
            value[i].update({lineIn[0].split()[j]:lineIn[i+1].split()[j]})

print(value)
gate_type = {}

for node in pi:
     gate_type.update({node:"PI"})

for node in l:
     gate_type.update({node.output:node.g_type})
nx.set_node_attributes(g,gate_type,name = "gateType")

nl = list(nx.topological_sort(g))
print(g)
print('Nodes in topological order',nl)



[{'N1': '0', 'N2': '1', 'N3': '0', 'N6': '0', 'N7': '0'}, {'N1': '0', 'N2': '0', 'N3': '1', 'N6': '0', 'N7': '0'}, {'N1': '1', 'N2': '0', 'N3': '0', 'N6': '0', 'N7': '0'}, {'N1': '0', 'N2': '0', 'N3': '1', 'N6': '1', 'N7': '1'}, {'N1': '1', 'N2': '1', 'N3': '1', 'N6': '1', 'N7': '1'}, {'N1': '1', 'N2': '1', 'N3': '1', 'N6': '0', 'N7': '0'}, {'N1': '1', 'N2': '1', 'N3': '1', 'N6': '1', 'N7': '0'}, {'N1': '1', 'N2': '1', 'N3': '0', 'N6': '0', 'N7': '0'}, {'N1': '0', 'N2': '1', 'N3': '1', 'N6': '0', 'N7': '1'}, {'N1': '0', 'N2': '0', 'N3': '1', 'N6': '1', 'N7': '0'}]
DiGraph with 11 nodes and 12 edges
Nodes in topological order ['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23']


Solving using the topological order

In [5]:

Graph = g.nodes(data=True)

typemap= {}
for item in Graph:
    typemap.update({item[0]:item[1]["gateType"]})

for i in range(len(lineIn)-1):   
   
   for j in range(len(pi),len(nl)):
      p = list(g.predecessors(nl[j]))
      out = 0 
      
      if typemap[nl[j]] == "nand2":
         a =  int(value[i][p[0]]) * int(value[i][p[1]])
         if a == 0 :
            out = 1
            
      if typemap[nl[j]] == "and2":
         out =  p[0] * p[1]

      if typemap[nl[j]] == "nor2":
         if p[0] + p[1] < 1:
            out = 1
      
      if typemap[nl[j]] == "or2":
         if p[0] + p[1] > 1:
            out = 1

      if typemap[nl[j]] == "inv":
         if p[0] == 0:
            out = 1

      if typemap[nl[j]] == "xnor2":
         if p[0] == p[1]:
            out = 1

      if typemap[nl[j]] == "or2":
         if p[0] != p[1]:
            out = 1
      value[i].update({nl[j]:out})
print(value[0])
           

{'N1': '0', 'N2': '1', 'N3': '0', 'N6': '0', 'N7': '0', 'n_0': 1, 'n_1': 1, 'n_3': 0, 'n_2': 1, 'N22': 1, 'N23': 1}


AND2, OR2, NOT, NOR2, NAND2, XOR2, XNOR2

In [21]:
print(value)

[{'N1': '0', 'N2': '1', 'N3': '0', 'N6': '0', 'N7': '0'}, {'N1': '0', 'N2': '0', 'N3': '1', 'N6': '0', 'N7': '0'}, {'N1': '1', 'N2': '0', 'N3': '0', 'N6': '0', 'N7': '0'}, {'N1': '0', 'N2': '0', 'N3': '1', 'N6': '1', 'N7': '1'}, {'N1': '1', 'N2': '1', 'N3': '1', 'N6': '1', 'N7': '1'}, {'N1': '1', 'N2': '1', 'N3': '1', 'N6': '0', 'N7': '0'}, {'N1': '1', 'N2': '1', 'N3': '1', 'N6': '1', 'N7': '0'}, {'N1': '1', 'N2': '1', 'N3': '0', 'N6': '0', 'N7': '0'}, {'N1': '0', 'N2': '1', 'N3': '1', 'N6': '0', 'N7': '1'}, {'N1': '0', 'N2': '0', 'N3': '1', 'N6': '1', 'N7': '0'}]
